<a href="https://colab.research.google.com/github/JoaquinGonzalezSimon/Data_science_and_ML_from_Medium/blob/main/230102_Restaurant_Recommendation_System_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This project was taken from the following link

https://thecleverprogrammer.com/2022/07/26/restaurant-recommendation-system-using-python/

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import files
import io
import zipfile

In [3]:
upload = files.upload()

Saving Tripadvisor_restaurant_recommendation.zip to Tripadvisor_restaurant_recommendation.zip


In [4]:
z = zipfile.ZipFile(io.BytesIO(upload['Tripadvisor_restaurant_recommendation.zip']))
z.namelist()

['TripAdvisor_RestauarantRecommendation.csv']

In [5]:
filename = z.namelist()[0]
data = pd.read_csv(z.open(filename))

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3062 entries, 0 to 3061
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              3062 non-null   object
 1   Street Address    3062 non-null   object
 2   Location          3062 non-null   object
 3   Type              3049 non-null   object
 4   Reviews           3062 non-null   object
 5   No of Reviews     3062 non-null   object
 6   Comments          2447 non-null   object
 7   Contact Number    3062 non-null   object
 8   Trip_advisor Url  3062 non-null   object
 9   Menu              3062 non-null   object
 10  Price_Range       3062 non-null   object
dtypes: object(11)
memory usage: 263.3+ KB


In [7]:
data.isnull().sum()

Name                  0
Street Address        0
Location              0
Type                 13
Reviews               0
No of Reviews         0
Comments            615
Contact Number        0
Trip_advisor Url      0
Menu                  0
Price_Range           0
dtype: int64

In [8]:
cols = [i.lower().replace(' ', '_') for i in data.columns]
data.columns = cols

In [9]:
data.head()

,name,street_address,location,type,reviews,no_of_reviews,comments,contact_number,trip_advisor_url,menu,price_range
0,Betty Lou's Seafood and Grill,318 Columbus Ave,"San Francisco, CA 94133-3908","Seafood, Vegetarian Friendly, Vegan Options",4.5 of 5 bubbles,243 reviews,NaN,+1 415-757-0569,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
1,Coach House Diner,55 State Rt 4,"Hackensack, NJ 07601-6337","Diner, American, Vegetarian Friendly",4 of 5 bubbles,84 reviews,"Both times we were there very late, after 11 P...",+1 201-488-4999,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
2,Table Talk Diner,2521 South Rd Ste C,"Poughkeepsie, NY 12601-5476","American, Diner, Vegetarian Friendly",4 of 5 bubbles,256 reviews,Waitress was very friendly but a little pricey...,+1 845-849-2839,https://www.tripadvisor.com//Restaurant_Review...,http://tabletalkdiner.com/menu/breakfast/,$$ - $$$
3,Sixty Vines,3701 Dallas Pkwy,"Plano, TX 75093-7777","American, Wine Bar, Vegetarian Friendly",4.5 of 5 bubbles,235 reviews,Not sure why I went there for the second time....,+1 469-620-8463,https://www.tripadvisor.com//Restaurant_Review...,https://sixtyvines.com/menu/plano-tx/,$$ - $$$
4,The Clam Bar,3914 Brewerton Rd,"Syracuse, NY 13212","American, Bar, Seafood",4 of 5 bubbles,285 reviews,Doesn't look like much from the outside but wa...,+1 315-458-1662,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$


In [10]:
data.dropna(inplace=True)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2440 entries, 1 to 3061
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              2440 non-null   object
 1   street_address    2440 non-null   object
 2   location          2440 non-null   object
 3   type              2440 non-null   object
 4   reviews           2440 non-null   object
 5   no_of_reviews     2440 non-null   object
 6   comments          2440 non-null   object
 7   contact_number    2440 non-null   object
 8   trip_advisor_url  2440 non-null   object
 9   menu              2440 non-null   object
 10  price_range       2440 non-null   object
dtypes: object(11)
memory usage: 228.8+ KB


In [12]:
feature = data['type'].tolist()
feature

[' Diner, American, Vegetarian Friendly',
 ' American, Diner, Vegetarian Friendly',
 ' American, Wine Bar, Vegetarian Friendly',
 ' American, Bar, Seafood',
 ' American, Steakhouse, Bar',
 ' Mediterranean, Greek, Healthy',
 ' Italian, Pizza, Vegetarian Friendly',
 ' American, Steakhouse, Gluten Free Options',
 ' Italian, Vegetarian Friendly, Gluten Free Options',
 ' Cafe, American, Vegetarian Friendly',
 ' Irish, Bar, Pub',
 ' American, Bar, Vegetarian Friendly',
 ' American, Cafe, Healthy',
 ' American, Bar, Pub',
 ' Chinese, Asian, Taiwanese',
 ' Belgian, French, Cafe',
 ' Steakhouse, Mexican, Vegetarian Friendly',
 ' American, Vegetarian Friendly, Vegan Options',
 ' Bar, Contemporary, Fusion',
 ' American, European, Vegetarian Friendly',
 ' Italian, Vegetarian Friendly, Gluten Free Options',
 ' Latin, Spanish, Vegetarian Friendly',
 ' American, Bar, Gastropub',
 ' Italian, Vegetarian Friendly, Gluten Free Options',
 ' Spanish, Mediterranean, Wine Bar',
 ' Chinese, Asian, Vegetarian 

In [13]:
tfidf = text.TfidfVectorizer(input=feature, stop_words='english')
tfidf_matrix = tfidf.fit_transform(feature)
similarity = cosine_similarity(tfidf_matrix)

In [14]:
indices = pd.Series(data.index, index=data['name']).drop_duplicates()

In [15]:
def restaurant_recomm(name, similarity=similarity):
  index = indices[name]
  similarity_score = list(enumerate(similarity[index]))
  similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
  similarity_score = similarity_score[0:10]
  rest_indices = [i[0] for i in similarity_score]
  return data['name'].iloc[rest_indices]

In [17]:
print(restaurant_recomm('Market Grill'))

63                     Bisque Restaurant
90                   Jack's Oyster House
257             Christopher's Restaurant
493                    Truluck's Seafood
514             Duke's Seafood & Chowder
678                       The Fish Friar
886                     Knife & Fork Inn
975                      London Lennie's
1150    Spadafora's Down East Restaurant
1236                    KEE Oyster House
Name: name, dtype: object


In [29]:
test = data['name'][15]

In [30]:
index = indices[test]
sim_score = list(enumerate(similarity[index]))
sim_score = sorted(sim_score, reverse=True, key=lambda x: x[1])
sim_score = sim_score[0:10]
rest_ind = [i[0] for i in sim_score]
data['name'].iloc[rest_ind]

18                      Ravenous
1304     La Frite Belgian Bistro
2921                 Mimi's Cafe
1991          Creperie de la Mer
1596         Cannelle Patisserie
492     Shore Good Pancake House
481       Boucherie West Village
551       Boucherie West Village
1906      Boucherie West Village
1949                Savor Bistro
Name: name, dtype: object